In [1]:
from accounts.models import Administrator

ModuleNotFoundError: No module named 'accounts'

In [2]:
Gen = Administrator.objects.get(id=1)

NameError: name 'Administrator' is not defined

In [3]:
isinstance(Gen, Administrator)

NameError: name 'Gen' is not defined

In [4]:
Gen.email

NameError: name 'Gen' is not defined

In [5]:
Gen.employee_class

NameError: name 'Gen' is not defined

## Spending 2

In [6]:
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import calendar


class RecurringSpendingItem:
    """
    Represents a recurring spending item with attributes such as name, frequency, due date, amount, and optional end date.
    Provides methods to calculate funding status, generate future due dates, and debug information.
    """

    def __init__(self, name, frequency, due_date, amount, end_date=None, discretionary=False):
        """
        Initializes a RecurringSpendingItem instance.

         Args:
             name (str): The name or description of the spending item.
             frequency (str): The frequency of payments, one of 'bimonthly', 'monthly', 'yearly', or 'weekly'.
             due_date (str): The initial due date in the format '%Y-%m-%d'.
             amount (float): The total amount of the spending item.
             end_date (str, optional): The optional end date in the format '%Y-%m-%d'.

         Returns:
             None
         """
        self.name = name
        self.frequency = frequency
        self.due_date = datetime.strptime(due_date, "%Y-%m-%d")
        self.amount = amount
        self.end_date = datetime.strptime(end_date, "%Y-%m-%d") if end_date else None
        self.discretionary = discretionary
        self.spending_history = []

        if self.frequency == "bimonthly":
            self.current_timedelta = timedelta(days=15)
        elif self.frequency == "monthly":
            self.current_timedelta = relativedelta(months=1)
        elif self.frequency == "yearly":
            self.current_timedelta = relativedelta(years=1)
        elif self.frequency == "weekly":
            self.current_timedelta = timedelta(weeks=1)

    def generate_future_due_dates(self, target_date):
        """
        Generates future due dates for the spending item.

         Args:
             target_date (datetime): The target date for generating future due dates.

         Returns:
             list: A list of future due dates in the format '%Y-%m-%d'.
         """
        future_due_dates = []
        current_date = self.due_date

        # Calculate the upper limit for generating future due dates
        upper_limit = target_date + 5 * self.current_timedelta
        
        while current_date <= upper_limit:
            # Check if end_date is reached
            if self.end_date and current_date > self.end_date:
                break

            future_due_dates.append(current_date.strftime("%Y-%m-%d"))

            if self.frequency == "bimonthly":
                current_date += timedelta(days=15)
            elif self.frequency == "monthly":
                current_date += self.get_next_month_timedelta(current_date)
            elif self.frequency == "yearly":
                current_date += relativedelta(years=1)
            elif self.frequency == "weekly":
                current_date += timedelta(weeks=1)

        return future_due_dates
    
    def last_next_due_date(self, target_date):
        future_due_dates = self.generate_future_due_dates(target_date)
        future_due_dates = [datetime.strptime(date, "%Y-%m-%d") for date in future_due_dates]
        
        if target_date >= self.due_date:
            last_due_date = max([date for date in future_due_dates if date <= target_date])
            next_due_date = min([date for date in future_due_dates if date > target_date])
        
        return last_due_date, next_due_date


    def funding_status(self, target_date):
        """
        Calculates the funding status for the spending item.

         Args:
             target_date (datetime): The target date for calculating funding status.

         Returns:
             tuple: A tuple containing the accrued funding and remaining funding.
         """
        if self.end_date and target_date > self.end_date:
            return 0, 0  # Both accrued and remaining funding are 0 after the end date
            
        if target_date >= self.due_date:
            last_due_date, next_due_date = self.last_next_due_date(target_date)
            
            days_elapsed = (target_date - last_due_date).days
            days_in_period = (next_due_date - last_due_date).days
    
            if days_in_period < 0:
                daily_funding_required = self.amount
            else:
                daily_funding_required = self.amount / days_in_period
    
            days_remaining = (next_due_date - target_date).days
            remaining_funding = daily_funding_required * days_remaining
            accrued_funding = daily_funding_required * days_elapsed
    
            return accrued_funding, remaining_funding
    
        return 0, 0  # Both accrued and remaining funding are 0 before the next due date

    def annualized_amount(self, target_date):
        
        if self.end_date is None:
            # Calculate annualized amount based on frequency
            if self.frequency == "monthly":
                return self.amount * 12  # Expense occurs every month
            elif self.frequency == "quarterly":
                return self.amount * 4  # Expense occurs every quarter
            elif self.frequency == "yearly":
                return self.amount  # Expense occurs once a year
            elif self.frequency == "bimonthly":
                return self.amount * 26  # Expense occurs every two months (26 times a year)
            elif self.frequency == "weekly":
                return self.amount * 52  # Expense occurs every two months (26 times a year)
            # Add more frequency options as needed
            else:
                raise ValueError("Invalid frequency")

        else:
            # Calculate annualized amount based on actual expenses within the year
            # today = datetime.now().date()
            today = target_date
            start_date = max(self.due_date.date(), datetime(today.year, 1, 1).date())
            end_date = min(self.end_date.date(), datetime(today.year, 12, 31).date())
            
            if start_date > end_date:
                # No overlap between start and end date
                return 0
            
            days_between = (end_date - start_date).days + 1
            daily_amount = self.amount / days_between
            annualized_amount = daily_amount * 365

            return annualized_amount

    def debug_info(self, target_date):
        """
        Generates debugging information for the spending item's funding status.

         Args:
             target_date (datetime): The target date for debugging information.

         Returns:
             dict: A dictionary containing debug information.
         """
        # future_due_dates = self.generate_future_due_dates(target_date)
        # future_due_dates = [datetime.strptime(date, "%Y-%m-%d") for date in future_due_dates]
    
        if target_date >= self.due_date:
            last_due_date, next_due_date = self.last_next_due_date(target_date)
    
            days_elapsed = (target_date - last_due_date).days
            days_in_period = (next_due_date - last_due_date).days
    
            if days_in_period == 0:
                daily_funding_required = self.amount
            else:
                daily_funding_required = self.amount / days_in_period
    
            days_remaining = (next_due_date - target_date).days
            remaining_funding = daily_funding_required * days_remaining
            accrued_funding = daily_funding_required * days_elapsed
    
            return {
                "Name": self.name,
                "Frequency": self.frequency,
                "Due Date": self.due_date.strftime('%Y-%m-%d'),
                "Amount": self.amount,
                "Last Due Date": last_due_date.strftime('%Y-%m-%d'),
                "Target Date": target_date.strftime('%Y-%m-%d'),
                "Next Due Date": next_due_date.strftime('%Y-%m-%d'),
                "Days Elapsed": days_elapsed,
                "Days in Period": days_in_period,
                "Daily Funding Required": daily_funding_required,
                "Accrued Funding": accrued_funding,
                "Remaining Funding": remaining_funding
            }
        else:
            return {
                "Name": self.name,
                "Frequency": self.frequency,
                "Due Date": self.due_date.strftime('%Y-%m-%d'),
                "Amount": self.amount,
                "Target Date": target_date.strftime('%Y-%m-%d'),
                "Next Due Date": "",
                "Last Due Date": "",
                "Days Elapsed": 0,
                "Days in Period": 0,
                "Daily Funding Required": 0,
                "Accrued Funding": 0,
                "Remaining Funding": 0
            }

    def get_next_month_timedelta(self, current_date):
        """
        Calculates the timedelta for the next month relative to a given date.

         Args:
             current_date (datetime): The current date.

         Returns:
             timedelta: The timedelta for the next month.
         """
        next_month = (current_date.month % 12) + 1
        days_in_next_month = calendar.monthrange(current_date.year, next_month)[1]
        return timedelta(days=days_in_next_month)

    def add_spending_record(self, date, amount_spent):
        """
        Adds a spending record to the history.

         Args:
             date (str): The date of the spending record in the format '%Y-%m-%d'.
             amount_spent (float): The amount spent.

         Returns:
             None
         """
        self.spending_history.append({
            'date': datetime.strptime(date, "%Y-%m-%d"),
            'amount_spent': amount_spent
        })

    def get_total_spent_between_dates(self, target_date):
        """
        Calculates the sum of amounts spent between two dates.

         Args:
             start_date (datetime): The start date for calculating the sum.
             end_date (datetime): The end date for calculating the sum.

         Returns:
             float: The sum of amounts spent between the start and end dates.
         """
        
        last_due_date, next_due_date = self.last_next_due_date(target_date)
            
        total_spent = 0
        for record in self.spending_history:
            if last_due_date <= record['date'] <= target_date:
                total_spent += record['amount_spent']
        return total_spent

    
    def compare_accrued_vs_actual(self, target_date):
        """
        Compares accrued funding with actual spending for discretionary spending items.

        Args:
            target_date (datetime): The target date for the comparison.

        Returns:
            dict: A dictionary containing the comparison results.
         """
        if not self.discretionary:
            return {"error": "This method is only applicable to discretionary spending items."}

        # Calculate accrued funding
        accrued_funding, _ = self.funding_status(target_date)

        # Calculate total actual spending
        total_actual_spending = self.get_total_spent_between_dates(target_date)

        # Calculate the difference
        difference = accrued_funding - total_actual_spending

        # last_due_date, next_due_date = 
        comparison_results = {
            "Name": self.name,
            "Frequency": self.frequency,
            "Last due date": self.last_next_due_date(target_date)[0].strftime('%Y-%m-%d'),
            "Next due date": self.last_next_due_date(target_date)[1].strftime('%Y-%m-%d'),
            "Amount": self.amount,
            "Accrued Funding": accrued_funding,
            "Total Actual Spending": total_actual_spending,
            "Difference": difference,
        }

        return comparison_results



#     def visualize_comparison_bar_plot(self, target_date):
        """
        Visualizes a stacked bar plot comparing accrued funding, unaccrued amount, and total actual spending
        for discretionary spending items.

        Args:
            target_date (datetime): The target date for visualization.

        Returns:
            None
        """
        if not self.discretionary:
            print("This method is only applicable to discretionary spending items.")
            return

        # Calculate accrued funding
        accrued_funding, _ = self.funding_status(target_date)

        # Calculate total actual spending
        total_actual_spending = self.get_total_spent_between_dates(target_date)

        # Calculate the percentage of accrued funding
        percentage_accrued_funding = (accrued_funding / self.amount) * 100

        # Calculate the unaccrued amount
        unaccrued_amount = self.amount - accrued_funding

        # Create a stacked bar plot
        categories = [self.name]
        bottom_values = [accrued_funding]
        top_values = [unaccrued_amount]
        total_actual_spending_values = [total_actual_spending]

        x = np.arange(len(categories))  # the label locations
        width = 0.35  # the width of the bars

        fig, ax = plt.subplots(figsize=(8, 6))
        bars1 = ax.bar(x, bottom_values, width, label='Accrued Funding', color='b')
        bars2 = ax.bar(x, top_values, width, bottom=bottom_values, label='Unaccrued Amount', color='lightblue')
        bars3 = ax.bar(x + width, total_actual_spending_values, width, label='Total Actual Spending', color='g')

        # Add labels, title, and legend
        ax.set_xlabel('Items')
        ax.set_ylabel('Amount')
        ax.set_title('Accrued Funding vs. Unaccrued Amount vs. Total Actual Spending')
        ax.set_xticks(x)
        ax.set_xticklabels(categories)
        ax.legend()

        # Display the plot
        plt.tight_layout()
        plt.show()



    def visualize_difference(self, target_date):
        """
        Visualizes the difference between spending limit and total spent over time.
    
        Args:
            target_date (datetime): The target date for visualization.
    
        Returns:
            None
        """
        last_due_date, _ = self.last_next_due_date(target_date)
    
        # Generate a list of dates between last_due_date and target_date
        date_range = [last_due_date + timedelta(days=i) for i in range((target_date - last_due_date).days + 1)]
    
        # Calculate accrued funding and total spent for each date
        accrued_funding = []
        total_spent = []
    
        for date in date_range:
            funding, spent = self.funding_status(date)
            accrued_funding.append(funding)
            total_spent.append(self.get_total_spent_between_dates(date))
    
        # Calculate the difference between accrued funding and total spent
        difference = [spent - funding for funding, spent in zip(accrued_funding, total_spent)]
    
        # Create a bar chart to visualize the data (unstacked)
        x = np.arange(len(date_range))
        width = 0.25
    
        fig, ax = plt.subplots(figsize=(12, 6))
        bars1 = ax.bar(x - width, accrued_funding, width, label='Accrued Funding')
        bars2 = ax.bar(x, total_spent, width, label='Total Spent')
        bars3 = ax.plot(x + width, difference, width, label='Difference')
    
        ax.set_xlabel('Date')
        ax.set_ylabel('Amount')
        ax.set_title(f'Spending Comparison Over Time for {self.name}')
        ax.set_xticks(x)
        

        # Remove the top and right spines (contour lines)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['bottom'].set_position('zero')
        ax.set_xticklabels([date.strftime('%Y-%m-%d') for date in date_range], rotation=45)

        # Adjust y-axis limits to keep the x-axis labels at the bottom
        
#         ax.legend()
    
#         plt.tight_layout()
    
#         # Show the plot or save it to a file
#         plt.show()

    
    
#     def __str__(self):
#         """
#         Returns a string representation of the spending item.

#         Returns:
#             str: A string representation of the spending item.
#         """
#         return f"{self.name} - Frequency: {self.frequency}, Due Date: {self.due_date.strftime('%Y-%m-%d')}, Amount: ${self.amount}" 

class RecurringSpendingItem:
    """
    Represents a recurring spending item with attributes such as name, frequency, due date, amount, and optional end date.
    Provides methods to calculate funding status, generate future due dates, and debug information.

    Attributes:
        name (str): The name or description of the spending item.
        frequency (str): The frequency of payments, one of 'bimonthly', 'monthly', 'yearly', or 'weekly'.
        due_date (datetime): The initial due date.
        amount (float): The total amount of the spending item.
        end_date (datetime, optional): The optional end date.
        discretionary (bool): Whether the spending item is discretionary.

    Methods:
        generate_future_due_dates(target_date):
            Generates future due dates for the spending item.

        last_next_due_date(target_date):
            Returns the last due date and the next due date relative to a target date.

        funding_status(target_date):
            Calculates the funding status for the spending item.

        annualized_amount(target_date):
            Calculates the annualized amount for the spending item.

        debug_info(target_date):
            Generates debugging information for the spending item's funding status.

        get_next_month_timedelta(current_date):
            Calculates the timedelta for the next month relative to a given date.

        add_spending_record(date, amount_spent):
            Adds a spending record to the history.

        get_total_spent_between_dates(target_date):
            Calculates the sum of amounts spent between two dates.

        compare_accrued_vs_actual(target_date):
            Compares accrued funding with actual spending for discretionary spending items.

        visualize_comparison_bar_plot(target_date):
            Visualizes a stacked bar plot comparing accrued funding, unaccrued amount, and total actual spending.

        visualize_difference(target_date):
            Visualizes the difference between spending limit and total spent over time.

        __str__():
            Returns a string representation of the spending item.
    """

    def __init__(self, name, frequency, due_date, amount, end_date=None, discretionary=False):
        """
        Initializes a RecurringSpendingItem instance.

        Args:
            name (str): The name or description of the spending item.
            frequency (str): The frequency of payments, one of 'bimonthly', 'monthly', 'yearly', or 'weekly'.
            due_date (str): The initial due date in the format '%Y-%m-%d'.
            amount (float): The total amount of the spending item.
            end_date (str, optional): The optional end date in the format '%Y-%m-%d'.

        Returns:
            None
        """
        self.name = name
        self.frequency = frequency
        self.due_date = datetime.strptime(due_date, "%Y-%m-%d")
        self.amount = amount
        self.end_date = datetime.strptime(end_date, "%Y-%m-%d") if end_date else None
        self.discretionary = discretionary
        self.spending_history = []

        if self.frequency == "bimonthly":
            self.current_timedelta = timedelta(days=15)
        elif self.frequency == "monthly":
            self.current_timedelta = relativedelta(months=1)
        elif self.frequency == "yearly":
            self.current_timedelta = relativedelta(years=1)
        elif self.frequency == "weekly":
            self.current_timedelta = timedelta(weeks=1)
            
    def generate_future_due_dates(self, target_date):
        """
        Generates future due dates for the spending item.

        Args:
            target_date (datetime): The target date for generating future due dates.

        Returns:
            list: A list of future due dates in the format '%Y-%m-%d'.
        """
        future_due_dates = []
        current_date = self.due_date

        # Calculate the upper limit for generating future due dates
        upper_limit = target_date + 5 * self.current_timedelta
        
        while current_date <= upper_limit:
            # Check if end_date is reached
            if self.end_date and current_date > self.end_date:
                break

            future_due_dates.append(current_date.strftime("%Y-%m-%d"))

            if self.frequency == "bimonthly":
                current_date += timedelta(days=15)
            elif self.frequency == "monthly":
                current_date += self.get_next_month_timedelta(current_date)
            elif self.frequency == "yearly":
                current_date += relativedelta(years=1)
            elif self.frequency == "weekly":
                current_date += timedelta(weeks=1)

        return future_due_dates
        
    def last_next_due_date(self, target_date):
        """
        Returns the last due date and the next due date relative to a target date.

        Args:
            target_date (datetime): The target date for which due dates should be determined.

        Returns:
            tuple: A tuple containing the last due date and the next due date.
        """
        future_due_dates = self.generate_future_due_dates(target_date)
        future_due_dates = [datetime.strptime(date, "%Y-%m-%d") for date in future_due_dates]
        
        if target_date >= self.due_date:
            last_due_date = max([date for date in future_due_dates if date <= target_date])
            next_due_date = min([date for date in future_due_dates if date > target_date])
        
        return last_due_date, next_due_date
        
    def funding_status(self, target_date):
        """
        Calculates the funding status for the spending item.

        Args:
            target_date (datetime): The target date for calculating funding status.

        Returns:
            tuple: A tuple containing the accrued funding and remaining funding.
        """
        if self.end_date and target_date > self.end_date:
            return 0, 0  # Both accrued and remaining funding are 0 after the end date
            
        if target_date >= self.due_date:
            last_due_date, next_due_date = self.last_next_due_date(target_date)
            
            days_elapsed = (target_date - last_due_date).days
            days_in_period = (next_due_date - last_due_date).days
    
            if days_in_period < 0:
                daily_funding_required = self.amount
            else:
                daily_funding_required = self.amount / days_in_period
    
            days_remaining = (next_due_date - target_date).days
            remaining_funding = daily_funding_required * days_remaining
            accrued_funding = daily_funding_required * days_elapsed
    
            return accrued_funding, remaining_funding
    
        return 0, 0  # Both accrued and remaining funding are 0 before the next due date

    def annualized_amount(self, target_date):
        """
        Calculates the annualized amount for the spending item.

        Args:
            target_date (datetime): The target date for which the annualized amount should be calculated.

        Returns:
            float: The annualized amount.
        """
        if self.end_date is None:
            # Calculate annualized amount based on frequency
            if self.frequency == "monthly":
                return self.amount * 12  # Expense occurs every month
            elif self.frequency == "quarterly":
                return self.amount * 4  # Expense occurs every quarter
            elif self.frequency == "yearly":
                return self.amount  # Expense occurs once a year
            elif self.frequency == "bimonthly":
                return self.amount * 26  # Expense occurs every two months (26 times a year)
            elif self.frequency == "weekly":
                return self.amount * 52  # Expense occurs every two months (26 times a year)
            # Add more frequency options as needed
            else:
                raise ValueError("Invalid frequency")

        else:
            # Calculate annualized amount based on actual expenses within the year
            # today = datetime.now().date()
            today = target_date
            start_date = max(self.due_date.date(), datetime(today.year, 1, 1).date())
            end_date = min(self.end_date.date(), datetime(today.year, 12, 31).date())
            
            if start_date > end_date:
                # No overlap between start and end date
                return 0
            
            days_between = (end_date - start_date).days + 1
            daily_amount = self.amount / days_between
            annualized_amount = daily_amount * 365

            return annualized_amount

    def debug_info(self, target_date):
        """
        Generates debugging information for the spending item's funding status.

        Args:
            target_date (datetime): The target date for debugging information.

        Returns:
            dict: A dictionary containing debug information.
        """
        if target_date >= self.due_date:
            last_due_date, next_due_date = self.last_next_due_date(target_date)
    
            days_elapsed = (target_date - last_due_date).days
            days_in_period = (next_due_date - last_due_date).days
    
            if days_in_period == 0:
                daily_funding_required = self.amount
            else:
                daily_funding_required = self.amount / days_in_period
    
            days_remaining = (next_due_date - target_date).days
            remaining_funding = daily_funding_required * days_remaining
            accrued_funding = daily_funding_required * days_elapsed
    
            return {
                "Name": self.name,
                "Frequency": self.frequency,
                "Due Date": self.due_date.strftime('%Y-%m-%d'),
                "Amount": self.amount,
                "Last Due Date": last_due_date.strftime('%Y-%m-%d'),
                "Target Date": target_date.strftime('%Y-%m-%d'),
                "Next Due Date": next_due_date.strftime('%Y-%m-%d'),
                "Days Elapsed": days_elapsed,
                "Days in Period": days_in_period,
                "Daily Funding Required": daily_funding_required,
                "Accrued Funding": accrued_funding,
                "Remaining Funding": remaining_funding
            }
        else:
            return {
                "Name": self.name,
                "Frequency": self.frequency,
                "Due Date": self.due_date.strftime('%Y-%m-%d'),
                "Amount": self.amount,
                "Target Date": target_date.strftime('%Y-%m-%d'),
                "Next Due Date": "",
                "Last Due Date": "",
                "Days Elapsed": 0,
                "Days in Period": 0,
                "Daily Funding Required": 0,
                "Accrued Funding": 0,
                "Remaining Funding": 0
            }

    def get_next_month_timedelta(self, current_date):
        """
        Calculates the timedelta for the next month relative to a given date.

        Args:
            current_date (datetime): The current date.

        Returns:
            timedelta: The timedelta for the next month.
        """
        next_month = (current_date.month % 12) + 1
        days_in_next_month = calendar.monthrange(current_date.year, next_month)[1]
        return timedelta(days=days_in_next_month)
        
    def add_spending_record(self, date, amount_spent):
        """
        Adds a spending record to the history.

        Args:
            date (str): The date of the spending record in the format '%Y-%m-%d'.
            amount_spent (float): The amount spent.

        Returns:
            None
        """
        self.spending_history.append({
            'date': datetime.strptime(date, "%Y-%m-%d"),
            'amount_spent': amount_spent
        })
        
    def get_total_spent_between_dates(self, target_date):
        """
        Calculates the sum of amounts spent between two dates.

        Args:
            target_date (datetime): The target date for calculating the sum.

        Returns:
            float: The sum of amounts spent between the start and end dates.
        """
        last_due_date, next_due_date = self.last_next_due_date(target_date)
            
        total_spent = 0
        for record in self.spending_history:
            if last_due_date <= record['date'] <= target_date:
                total_spent += record['amount_spent']
        return total_spent
        
    def compare_accrued_vs_actual(self, target_date):
        """
        Compares accrued funding with actual spending for discretionary spending items.

        Args:
            target_date (datetime): The target date for the comparison.

        Returns:
            dict: A dictionary containing the comparison results.
        """
        if not self.discretionary:
            return {"error": "This method is only applicable to discretionary spending items."}

        # Calculate accrued funding
        accrued_funding, _ = self.funding_status(target_date)

        # Calculate total actual spending
        total_actual_spending = self.get_total_spent_between_dates(target_date)

        # Calculate the difference
        difference = accrued_funding - total_actual_spending

        # last_due_date, next_due_date = 
        comparison_results = {
            "Name": self.name,
            "Frequency": self.frequency,
            "Last due date": self.last_next_due_date(target_date)[0].strftime('%Y-%m-%d'),
            "Next due date": self.last_next_due_date(target_date)[1].strftime('%Y-%m-%d'),
            "Amount": self.amount,
            "Accrued Funding": accrued_funding,
            "Total Actual Spending": total_actual_spending,
            "Difference": difference,
        }
        return comparison_results
        
    def visualize_comparison_bar_plot(self, target_date):
        """
        Visualizes a stacked bar plot comparing accrued funding, unaccrued amount, and total actual spending.

        Args:
            target_date (datetime): The target date for visualization.

        Returns:
            None
        """
        if not self.discretionary:
            print("This method is only applicable to discretionary spending items.")
            return

        # Calculate accrued funding
        accrued_funding, _ = self.funding_status(target_date)

        # Calculate total actual spending
        total_actual_spending = self.get_total_spent_between_dates(target_date)

        # Calculate the percentage of accrued funding
        percentage_accrued_funding = (accrued_funding / self.amount) * 100

        # Calculate the unaccrued amount
        unaccrued_amount = self.amount - accrued_funding

        # Create a stacked bar plot
        categories = [self.name]
        bottom_values = [accrued_funding]
        top_values = [unaccrued_amount]
        total_actual_spending_values = [total_actual_spending]

        x = np.arange(len(categories))  # the label locations
        width = 0.35  # the width of the bars

        fig, ax = plt.subplots(figsize=(8, 6))
        bars1 = ax.bar(x, bottom_values, width, label='Accrued Funding', color='b')
        bars2 = ax.bar(x, top_values, width, bottom=bottom_values, label='Unaccrued Amount', color='lightblue')
        bars3 = ax.bar(x + width, total_actual_spending_values, width, label='Total Actual Spending', color='g')

        # Add labels, title, and legend
        ax.set_xlabel('Items')
        ax.set_ylabel('Amount')
        ax.set_title('Accrued Funding vs. Unaccrued Amount vs. Total Actual Spending')
        ax.set_xticks(x)
        ax.set_xticklabels(categories)
        ax.legend()

        # Display the plot
        plt.tight_layout()
        plt.show()
        
    def visualize_difference(self, target_date):
        """
        Visualizes the difference between spending limit and total spent over time.

        Args:
            target_date (datetime): The target date for visualization.

        Returns:
            None
        """
            def visualize_difference(self, target_date):
        """
        Visualizes the difference between spending limit and total spent over time.
    
        Args:
            target_date (datetime): The target date for visualization.
    
        Returns:
            None
        """
        last_due_date, _ = self.last_next_due_date(target_date)
    
        # Generate a list of dates between last_due_date and target_date
        date_range = [last_due_date + timedelta(days=i) for i in range((target_date - last_due_date).days + 1)]
    
        # Calculate accrued funding and total spent for each date
        accrued_funding = []
        total_spent = []
    
        for date in date_range:
            funding, spent = self.funding_status(date)
            accrued_funding.append(funding)
            total_spent.append(self.get_total_spent_between_dates(date))
    
        # Calculate the difference between accrued funding and total spent
        difference = [spent - funding for funding, spent in zip(accrued_funding, total_spent)]
    
        # Create a bar chart to visualize the data (unstacked)
        x = np.arange(len(date_range))
        width = 0.25
    
        fig, ax = plt.subplots(figsize=(12, 6))
        bars1 = ax.bar(x - width, accrued_funding, width, label='Accrued Funding')
        bars2 = ax.bar(x, total_spent, width, label='Total Spent')
        bars3 = ax.plot(x + width, difference, width, label='Difference')
    
        ax.set_xlabel('Date')
        ax.set_ylabel('Amount')
        ax.set_title(f'Spending Comparison Over Time for {self.name}')
        ax.set_xticks(x)
        

        # Remove the top and right spines (contour lines)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['bottom'].set_position('zero')
        ax.set_xticklabels([date.strftime('%Y-%m-%d') for date in date_range], rotation=45)

        # Adjust y-axis limits to keep the x-axis labels at the bottom
        
        ax.legend()
    
        plt.tight_layout()
    
        # Show the plot or save it to a file
        plt.show()

    def __str__(self):
        """
        Returns a string representation of the spending item.

        Returns:
            str: A string representation of the spending item.
        """
        return f"{self.name} - Frequency: {self.frequency}, Due Date: {self.due_date.strftime('%Y-%m-%d')}, Amount: ${self.amount}" 


IndentationError: unexpected indent (2559226902.py, line 838)

In [7]:
# Create a discretionary spending item
discretionary_item = RecurringSpendingItem("Discretionary Item", "monthly", "2023-01-15", 500, end_date="2024-12-31", discretionary=True)

# Add spending records to the spending history
discretionary_item.add_spending_record("2023-01-20", 50)
discretionary_item.add_spending_record("2023-02-10", 60)
discretionary_item.add_spending_record("2023-03-05", 70)

# Compare accrued funding with actual spending for a specific target date
target_date = datetime.strptime("2023-03-07", "%Y-%m-%d")
comparison_results = discretionary_item.compare_accrued_vs_actual(target_date)

# Print the comparison results
print(comparison_results)
discretionary_item.visualize_comparison_bar_plot(target_date)

NameError: name 'RecurringSpendingItem' is not defined

## Portfolio

In [8]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Define the RecurringSpendingItem class (you can use the one you provided earlier)

# Define the LiabilityPortfolio class
import matplotlib.pyplot as plt
import numpy as np

class LiabilityPortfolio:
    """
    A class representing a portfolio of recurring spending items.

    Attributes:
        items (list): A list of recurring spending items in the portfolio.

    Methods:
        add_item(item):
            Adds a recurring spending item to the portfolio.

        add_spending_record(item_name, date, amount_spent):
            Adds a spending record to a specific item in the portfolio.

        get_item_by_name(item_name):
            Retrieves a specific item from the portfolio by its name.

        aggregate_funding(target_date):
            Calculates the total accrued funding for the portfolio up to a specified target date.

        detailed_funding(target_date):
            Calculates detailed funding information for each item in the portfolio up to a specified target date.

        visualize_accrued_funding_pie(target_date):
            Generates and displays a pie chart showing the distribution of accrued funding as of a target date.

        visualize_annualized_amount_pie(target_date):
            Generates and displays a pie chart showing the distribution of annualized amounts as of a target date.

        visualize_comparison_bar_plot(target_date):
            Generates and displays a stacked bar plot comparing accrued funding, unaccrued amount, and total actual spending
            for discretionary spending items as of a target date.
    """

    def __init__(self):
        """
        Initializes an empty LiabilityPortfolio.
        """
        self.items = []

    def add_item(self, item):
        """
        Adds a recurring spending item to the portfolio.

        Args:
            item (RecurringSpendingItem): The recurring spending item to be added to the portfolio.

        Raises:
            ValueError: If the item is not an instance of RecurringSpendingItem.
        """
        if isinstance(item, RecurringSpendingItem):
            self.items.append(item)
        else:
            raise ValueError("Item must be an instance of RecurringSpendingItem")

    def add_spending_record(self, item_name, date, amount_spent):
        """
        Adds a spending record to a specific item in the portfolio.

        Args:
            item_name (str): The name of the item to which the spending record should be added.
            date (str): The date of the spending record in the format '%Y-%m-%d'.
            amount_spent (float): The amount spent.

        Returns:
            bool: True if the spending record was added successfully, False if the item was not found.
        """
        item = self.get_item_by_name(item_name)
        if item:
            item.add_spending_record(date, amount_spent)
            return True
        else:
            return False

    def get_item_by_name(self, item_name):
        """
        Retrieves a specific item from the portfolio by its name.

        Args:
            item_name (str): The name of the item to retrieve.

        Returns:
            RecurringSpendingItem or None: The item with the specified name, or None if not found.
        """
        for item in self.items:
            if item.name == item_name:
                return item
        return None

    def aggregate_funding(self, target_date):
        """
        Calculates the total accrued funding for the portfolio up to a specified target date.

        Args:
            target_date (datetime): The target date for which accrued funding should be calculated.

        Returns:
            float: The total accrued funding up to the target date.
        """
        total_accrued_funding = sum(item.funding_status(target_date)[0] for item in self.items if item.end_date is None or target_date < item.end_date)
        return total_accrued_funding

    def detailed_funding(self, target_date):
        """
        Calculates detailed funding information for each item in the portfolio up to a specified target date.

        Args:
            target_date (datetime): The target date for which funding details should be calculated.

        Returns:
            dict: A dictionary containing funding details for each item, indexed by item name.
        """
        funding_details = {}
        for item in self.items:
            if target_date >= item.due_date and item.end_date is None:
                accrued_funding, remaining_funding = item.funding_status(target_date)
                last_due_date, next_due_date = item.last_next_due_date(target_date)

                funding_details[item.name] = {
                    "Accrued Funding": accrued_funding,
                    "Remaining Funding": remaining_funding,
                    "Last due date": last_due_date,
                    "Target date": target_date,
                    "Next due Date": next_due_date,
                    "Frequency": item.frequency
                }
            elif target_date >= item.due_date and item.end_date >= target_date:
                accrued_funding, remaining_funding = item.funding_status(target_date)
                last_due_date, next_due_date = item.last_next_due_date(target_date)

                funding_details[item.name] = {
                    "Accrued Funding": accrued_funding,
                    "Remaining Funding": remaining_funding,
                    "Last due date": last_due_date,
                    "Target date": target_date,
                    "Next due Date": next_due_date,
                    "Frequency": item.frequency
                }
        return funding_details

    def visualize_accrued_funding_pie(self, target_date):
        """
        Generates and displays a pie chart showing the distribution of accrued funding as of a target date.

        Args:
            target_date (datetime): The target date for which the pie chart should be generated and displayed.
        """
        # Get the Accrued Funding for each item
        funding_details = self.detailed_funding(target_date)

        # Extract item names and accrued funding amounts
        item_names = list(funding_details.keys())
        accrued_funding = [details["Accrued Funding"] for details in funding_details.values()]

        # Create a pie chart
        plt.figure(figsize=(8, 8))

        # Define a custom autopct function
        def label_function(pct):
            if pct < 1:
                return ""
            else:
                return f"{pct:.1f}%"

        # Use the custom autopct function and set pctdistance to move labels inside
        plt.pie(accrued_funding, labels=item_names, autopct=label_function, startangle=140, pctdistance=0.85)

        plt.title(f'Accrued Funding Distribution as of {target_date.strftime("%Y-%m-%d")}')

        # Show the pie chart
        plt.show()

    def visualize_annualized_amount_pie(self, target_date):
        """
        Generates and displays a pie chart showing the distribution of annualized amounts as of a target date.

        Args:
            target_date (datetime): The target date for which the pie chart should be generated and displayed.
        """
        # Calculate the annualized amount for each item
        annualized_amounts = [item.annualized_amount(target_date) for item in self.items]

        # Extract item names
        item_names = [item.name for item in self.items]

        # Calculate percentages and exclude items with less than 0.5% percentage
        total_amount = sum(annualized_amounts)
        percentages = [(amount / total_amount) * 100 for amount in annualized_amounts]

        filtered_items = []
        filtered_percentages = []

        for name, percent in zip(item_names, percentages):
            if percent >= 1:
                filtered_items.append(name)
                filtered_percentages.append(percent)

        # Create a pie chart with custom formatting for filtered items
        plt.figure(figsize=(8, 8))
        ax = plt.gca()
        ax.pie(filtered_percentages,
               labels=filtered_items,
               autopct='%1.1f%%',
               startangle=140,
               pctdistance=0.85,
               radius=0.8)  # Adjust radius here

        plt.title(f'Annualized Amount Distribution as of {target_date.strftime("%Y-%m-%d")}')

        plt.axis('equal')
        plt.show()

    def visualize_comparison_bar_plot(self, target_date):
        """
        Generates and displays a stacked bar plot comparing accrued funding, unaccrued amount, and total actual spending
        for discretionary spending items as of a target date.

        Args:
            target_date (datetime): The target date for which the stacked bar plot should be generated and displayed.
        """
        # Get discretionary spending items
        discretionary_items = [item for item in self.items if item.discretionary]

        if not discretionary_items:
            print("No discretionary spending items in the portfolio.")
            return

        # Calculate accrued funding, unaccrued amount, and total actual spending for each discretionary item
        categories = [item.name for item in discretionary_items]
        accrued_funding_values = []
        unaccrued_amount_values = []
        total_actual_spending_values = []

        for item in discretionary_items:
            accrued_funding, _ = item.funding_status(target_date)
            total_actual_spending = item.get_total_spent_between_dates(target_date)
            unaccrued_amount = item.amount - accrued_funding

            accrued_funding_values.append(accrued_funding)
            unaccrued_amount_values.append(unaccrued_amount)
            total_actual_spending_values.append(total_actual_spending)

        x = np.arange(len(categories))  # the label locations
        width = 0.35  # the width of the bars

        fig, ax = plt.subplots(figsize=(10, 6))
        bars1 = ax.bar(x, accrued_funding_values, width, label='Accrued Funding', color='b')
        bars2 = ax.bar(x, unaccrued_amount_values, width, bottom=accrued_funding_values, label='Unaccrued Amount', color='lightblue')
        bars3 = ax.bar(x + width, total_actual_spending_values, width, label='Total Actual Spending', color='g')

        # Add labels, title, and legend
        ax.set_xlabel('Items')
        ax.set_ylabel('Amount')
        ax.set_title('Accrued Funding vs. Unaccrued Amount vs. Total Actual Spending')
        ax.set_xticks(x)
        # ax.set_xticklabels(categories)
        ax.set_xticklabels(categories, rotation=90)
        ax.legend()

        # Display the plot
        plt.tight_layout()
        plt.show()



# Create a LiabilityPortfolio instance
portfolio = LiabilityPortfolio()

# Create RecurringSpendingItem instances and add them to the portfolio
item1 = RecurringSpendingItem("Rent", "monthly", "2023-09-30", 1000)
item2 = RecurringSpendingItem("Insurance", "yearly", "2022-12-15", 600)

portfolio.add_item(item1)
portfolio.add_item(item2)

# Set a target date for funding calculations
target_date = datetime(2023, 10, 15)

# Calculate the total accrued funding for the portfolio
total_funding = portfolio.aggregate_funding(target_date)
print(f"Total Accrued Funding on {target_date.strftime('%Y-%m-%d')}: ${total_funding}")

# Calculate detailed funding information for each item in the portfolio
detailed_funding = portfolio.detailed_funding(target_date)
for item_name, details in detailed_funding.items():
    print(f"{item_name}:")
    print(f"  Accrued Funding: ${details['Accrued Funding']}")
    print(f"  Remaining Funding: ${details['Remaining Funding']}")
    print(f"  Frequency: {details['Frequency']}")
    print(f"  Last due Date: {details['Last due date']}")
    print(f"  Last due Date: {details['Target date']}")
    print(f"  Next due Date: {details['Next due Date']}")
    


NameError: name 'RecurringSpendingItem' is not defined

In [9]:
spending_list = [("Hypothèque", "bimonthly", "2023-09-07", 451, None, False, "verified"),
                 ("Hydro", "monthly", "2023-09-07", 203, None, False, "verified"),
                 ("assurances pmt1", "monthly", "2022-08-28", 96.56, None, False, "verified"),
                 ("assurances pmt2", "monthly", "2022-09-01", 69.59, None, False, "verified"),
                 ("Auto pmt tangerine", "monthly", "2022-10-02", 50, None, False),
                 ("Internet", "monthly", "2022-08-15", 35, None, False, "verified"),
                 ("Netflix", "monthly", "2022-08-03", 7, None, False, 'verified'),
                 ("Frais garde", "monthly", "2022-10-01", 200, None, False),
                 ("Taxe scholaire", "yearly", "2022-03-01", 400, None, False, "verified"),
                 ("Taxe municipales 1", "yearly", "2022-03-01", 1400, None, False, "verified"),
                 ("Taxe municipales 2", "yearly", "2022-06-01", 1400, None, False, "verified"),
                 ("Immatriculation", "yearly", "2023-09-15", 278, None, False, "Verified"),
                 ("Permis de conduire", "yearly", "2023-08-07", 90, None, False, "Verified"),
                 
                 ("Finacement CELI", "bimonthly", "2022-10-01", 501, "2023-12-31", True, "verified"),
                 ("Finacement Voyage Maroc", "bimonthly", "2023-01-01", 400, "2024-03-01", True),
                 ("epicerie", "weekly", "2023-10-02", 150, None, True),
                 ("vin", "monthly", "2023-10-02", 260, None, True),
                 ("Cours Enfants", "yearly", "2022-09-11", 2000, None, False),
                 ("Estheticienne", "monthly", "2022-10-02", 80, None, True),
                 ("Cheveux", "monthly", "2022-10-02", 25, None, True),
                 ("vie sociale", "monthly", "2022-10-02", 150, None, True),
                 ("Restaurant", "yearly", "2022-09-11", 1000, None, True),
                 ("Cadeau", "yearly", "2022-09-11", 400, None, True),
                 ("Massages", "yearly", "2022-09-11", 240, None, True),
                 ("Vetement Enfants", "yearly", "2022-09-11", 300, None, True),
                 ("Vetement Genevieve", "yearly", "2022-09-11", 500, None, True),
                 ("Pharmacie", "yearly", "2022-09-11", 200, None, True),
                 ("imprévue", "monthly", "2023-10-02", 300, None, True)
                 ]



In [10]:
portfolio = LiabilityPortfolio()

for spending in spending_list:
    try:
        spending_item = RecurringSpendingItem(name = spending[0],
                                              frequency = spending[1], 
                                              due_date = spending[2],
                                              amount = spending[3],
                                              end_date = spending[4],
                                              discretionary = spending[5])
    except:
        spending_item = RecurringSpendingItem(name = spending[0],
                                              frequency = spending[1], 
                                              due_date = spending[2],
                                              amount = spending[3])
        
    portfolio.add_item(spending_item)
    

NameError: name 'RecurringSpendingItem' is not defined

In [11]:
# Set a target date for funding calculations
target_date = datetime(2023, 11, 12)

# Calculate the total accrued funding for the portfolio
total_funding = portfolio.aggregate_funding(target_date)
print(f"Total Accrued Funding on {target_date.strftime('%Y-%m-%d')}: ${total_funding}")

# Calculate detailed funding information for each item in the portfolio
# detailed_funding = portfolio.detailed_funding(target_date)
# for item_name, details in detailed_funding.items():
#     print(f"{item_name}:")
#     print(f"  Accrued Funding: ${details['Accrued Funding']}")
#     print(f"  Remaining Funding: ${details['Remaining Funding']}")
#     print(f"  Frequency: {details['Frequency']}")
#     print(f"  Last due Date: {details['Last due date']}")
#     print(f"  Target Date: {details['Target date']}")
#     print(f"  Next due Date: {details['Next due Date']}")

portfolio.visualize_comparison_bar_plot(target_date)

# portfolio.visualize_annualized_amount_pie(target_date)

Total Accrued Funding on 2023-11-12: $0
No discretionary spending items in the portfolio.
